In [1]:
## Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. One category of liquidated damages is related to chronic problems in any category of service at the same premises. A chronic problem is defined as more than one missed pickup for any address. The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

After determining the total amount of damages, you can look at other questions:

* What other types of complaints are there?
* Are there any geospatial analysis you can do?  Which visualizations can you create?
* How do metro crews compare to the contractor's performance?
* How much does each trash hauler owe?
* What were to total missed pickup by route?


SyntaxError: unterminated string literal (detected at line 3) (443242377.py, line 3)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df_trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [8]:
df_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


In [9]:
df_trash.describe()

,Request Number,Zip Code,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
count,20226.000000,20151.000000,20177.000000,2.019800e+04,2.019800e+04,20198.000000,20198.000000
mean,149178.986354,37180.621805,18.079100,1.763012e+06,6.595069e+05,-86.709972,36.120730
std,71837.306889,66.310919,9.917632,7.754951e+05,4.355405e+04,2.025630,0.845846
min,25270.000000,37013.000000,1.000000,1.663490e+06,-2.719106e+05,-87.031386,0.000322
25%,86052.750000,37205.000000,8.000000,1.727831e+06,6.406542e+05,-86.815290,36.090817
50%,145223.500000,37209.000000,19.000000,1.745845e+06,6.564858e+05,-86.754014,36.133888
75%,217162.750000,37214.000000,26.000000,1.759491e+06,6.756736e+05,-86.707973,36.186992
max,267137.000000,37228.000000,35.000000,3.496892e+07,2.204382e+06,-0.000798,36.466974


In [10]:
missing_values = df_trash.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
Request Number        0
Date Opened           0
Request               0
Description          31
Incident Address      9
Zip Code             75
Trash Hauler        901
Trash Route         947
Council District     49
State Plan X         28
State Plan Y         28
LONGITUDE            28
LATITUDE             28
dtype: int64


In [11]:
missed_pickups = df_trash[df_trash['Request'].isin(['Trash - Curbside/Alley Missed Pickup', 'Trash - Backdoor', 'Trash Collection Complaint'])]
print(f"Number of missed pickups: {len(missed_pickups)}")

Number of missed pickups: 19969


In [12]:
fine_amount = 200

In [13]:
misses_allowed = 1

In [14]:
#All missed per address
address_count = missed_pickups['Incident Address'].value_counts()

In [15]:
address_count

Incident Address
5135 Hickory Hollow Pkwy                                      21
3710 N NATCHEZ CT                                             20
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
6007 Obrien Ave, Nashville, TN 37209, United States           19
802 Crescent Rd, Nashville, TN 37205, United States           18
                                                              ..
310 Mckennell Dr, Nashville, TN 37206, United States           1
1238 Riverwood Dr, Nashville, TN 37216, United States          1
814 W OLD HICKORY BLVD                                         1
2831 BARCLAY DR                                                1
604 Somerset Ct, Nashville, TN 37217, United States            1
Name: count, Length: 13936, dtype: int64

In [16]:
#missed pickups for just Red River
red_river = missed_pickups[missed_pickups['Trash Hauler'] =='RED RIVER']
red_river

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220,267121,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed,"2709 Crestdale Dr, Nashville, TN 37214, United...",37214.0,RED RIVER,1502,15.0,1770240.199,676334.3993,-86.671860,36.189483
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637


In [17]:
red_river_counts = red_river['Incident Address'].value_counts()
red_river_counts

Incident Address
3710 N NATCHEZ CT                                             19
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
6007 Obrien Ave, Nashville, TN 37209, United States           18
5135 Hickory Hollow Pkwy                                      18
802 Crescent Rd, Nashville, TN 37205, United States           17
                                                              ..
909 Marilyn Rd, Nashville, TN 37209, United States             1
305 Harrison St, Nashville, TN 37211, United States            1
505 JOCELYN HOLLOW CT, 37205                                   1
744 Post Oak Dr, Nashville, TN 37013, United States            1
604 Somerset Ct, Nashville, TN 37217, United States            1
Name: count, Length: 9975, dtype: int64

In [18]:
# fines
all_fines = ((address_count[address_count > 1] - 1) * 200).sum()
all_fines

1204800

In [19]:
#fines just for Red River
red_river_fines = ((red_river_counts[red_river_counts > 1] - 1) * fine_amount).sum()
red_river_fines

844400

In [20]:
data_q4 = missed_pickups.copy()

In [21]:
# Convert dates to datetime
data_q4['Date Opened'] = pd.to_datetime(data_q4['Date Opened'])


In [22]:
# Sort by address and date
data_q4 = data_q4.sort_values(['Incident Address', 'Date Opened'])

In [23]:
# Use rolling window to count pickups within 180 days for each address
# This looks at each pickup and counts how many pickups happened in the last 180 days
count_in_window = (
    data_q4
    .groupby('Incident Address')
    .rolling(window='180d', on='Date Opened')
    ['Date Opened']
    .count()
)

# Count how many times an address had 3+ pickups in a 180-day window
times_with_3_or_more = (count_in_window >= 3).sum()


In [24]:
# Calculate total fines: $1500 for each occurrence
total_fines_q4 = times_with_3_or_more * 1500

In [25]:
print(total_fines_q4)

3271500


In [26]:
data_q4_alt2 = missed_pickups.copy()

In [28]:
import pandas as pd

In [29]:
def count_fines_no_reuse(group):
    dates = pd.to_datetime(group['Date Opened']).sort_values().to_list()

    fines = 0
    i = 0
    while i + 2 < len(dates):
        if (dates[i + 2] - dates[i]).days <= 180:
            fines += 1
            i += 3
        else:
            i += 1

    return fines





In [30]:
data_q4_alt2['Date Opened'] = pd.to_datetime(data_q4_alt2['Date Opened'])
data_q4_alt2 = data_q4_alt2.sort_values(['Incident Address', 'Date Opened'])



In [31]:
fines_per_address = (
    data_q4_alt2
        .groupby('Incident Address')
        .apply(count_fines_no_reuse)
)


In [32]:
print(fines_per_address.head())
total_fines = fines_per_address.sum() * 1500
print(total_fines)


Incident Address
, Nashville, TN , United States                             0
1 BELLE FORREST AVE C                                       0
10 Belle Forrest Ave                                        0
100 Bluefield Square, Nashville, TN 37214, United States    0
100 Braxton Hill Ct                                         0
dtype: int64
1824000
